# **LeNet**

<div align=center>
<img width="600" src="../image/5.5_lenet.png"/>
</div>
<div align=center>LeNet网络结构</div>

第一层卷积: (5, 5)的卷积核,步长为1,6个卷积核     
第一层池化: (2, 2)的池化,步长为2     
第二层卷积: (5, 5)的卷积核,步长为1,16个卷积核     
第二层池化: (2, 2)的池化,步长为2    

## **LeNet的pytorch实现**

In [8]:
import time
import torch
from torch import optim
from torch.nn import *

import sys
sys.path.append(r'C:\D\ProgramFile\jupyter\torch_learn\dive_to_dp\utils') 
import d2lzh as d2l
device = torch.device('cuda')

In [9]:
class LeNet(Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = Sequential(Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
                               Sigmoid(),
                               MaxPool2d(kernel_size=2, stride=2),
                               Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
                               Sigmoid(),
                               MaxPool2d(kernel_size=2, stride=2))
        self.fc = Sequential(Linear(16*4*4, 120),
                             Sigmoid(),
                             Linear(120, 84),
                             Sigmoid(),
                             Linear(84, 10))
    def forward(self, x):
        feature_map = self.conv(x)
        output = self.fc(feature_map.view(x.shape[0], -1))
        return output

In [10]:
net = LeNet()
print(net)

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


## **获取数据和训练模型**

In [11]:
batch_size = 256
train_iter, test_iter = d2l.load_fashion_mnist_data(batch_size)

In [12]:
# 支持gpu计算的评测函数
def evaluate_accuracy(data_iter, net, device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
    acc_sum, n = 0.0, 0.0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval() # 评测模式
                y_hat = net(X.to(device))
                acc_num = (y_hat.argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                acc_sum += acc_num
                net.train() # 训练模式
            elif ('is_training' in net.__code__.co_varnames):
                y_hat = net(X, is_train=False)
                acc_num = (y_hat.argmax(dim=1) == y).float().sum().item()
                acc_sum += acc_num
            else:
                y_hat = net(X)
                acc_num = (y_hat.argmax(dim=1) == y).float().sum().item()
                acc_sum += acc_num
            n += y.shape[0]
        return acc_sum / n

In [13]:
def train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
    net = net.to(device)
    print(f'training on {device}')
    loss = torch.nn.CrossEntropyLoss()
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0.0
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).float().sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print(train_acc_sum, train_l_sum)
        print(f'epoch{epoch+1}: loss {train_l_sum/batch_count:.4f} train_acc {train_acc_sum / n:.4f} test_acc {test_acc:.4f}')

In [14]:
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on cuda
20186.0 427.8613920211792
epoch1: loss 1.8207 train_acc 0.3364 test_acc 0.5806
39225.0 216.63814198970795
epoch2: loss 0.4609 train_acc 0.6538 test_acc 0.7052
43701.0 172.70168966054916
epoch3: loss 0.2450 train_acc 0.7284 test_acc 0.7387
44954.0 153.48643738031387
epoch4: loss 0.1633 train_acc 0.7492 test_acc 0.7535
45938.0 141.9695059657097
epoch5: loss 0.1208 train_acc 0.7656 test_acc 0.7638
